In [ ]:
!pip install -q transformers torch tqdm stanza emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 46.4 MB/s eta 0:00:00


In [ ]:
!pip install scikit-learn


In [ ]:
import json, re, unicodedata, emoji
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import stanza


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
stanza.download("zh")
nlp_zh = stanza.Pipeline(
    "zh",
    processors="tokenize,pos",
    use_gpu=torch.cuda.is_available()
)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Downloading default packages for language: zh-hans (Simplified_Chinese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/zh-hans/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:"zh" is an alias for "zh-hans"
INFO:stanza:Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package        |
------------------------------
| tokenize  | gsdsimp        |
| pos       | gsdsimp_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [ ]:
TRAIN_PATH = "/content/zho_laptop_train_alltasks.jsonl"
TEST_PATH  = "/content/zho_laptop_test_task2.jsonl"


In [ ]:
def load_jsonl(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
    return data

train_raw = load_jsonl(TRAIN_PATH)
test_raw  = load_jsonl(TEST_PATH)

print("Train reviews:", len(train_raw))
print("Test reviews :", len(test_raw))


Train reviews: 3490
Test reviews : 1000


In [ ]:
def clean_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = re.sub(r"\s+", " ", text).strip()
    return text


In [ ]:
def zh_tokens(text):
    doc = nlp_zh(text)
    return [w.text for s in doc.sentences for w in s.words]


In [ ]:
def build_bio(tokens, aspects, opinions):
    labels = ["O"] * len(tokens)

    def mark(span, tag):
        span_tokens = list(span)
        for i in range(len(tokens)):
            if tokens[i:i+len(span_tokens)] == span_tokens:
                labels[i] = f"B-{tag}"
                for j in range(1, len(span_tokens)):
                    labels[i+j] = f"I-{tag}"

    for a in aspects:
        mark(a, "ASP")
    for o in opinions:
        mark(o, "OPN")

    return labels


In [ ]:
def preprocess_train(data):
    samples = []
    for item in data:
        text = clean_text(item["Text"])
        tokens = zh_tokens(text)

        aspects, opinions = [], []
        for q in item["Quadruplet"]:
            if q["Aspect"] != "NULL":
                aspects.append(q["Aspect"])
                opinions.append(q["Opinion"])

        labels = build_bio(tokens, aspects, opinions)

        samples.append({
            "text": text,
            "tokens": tokens,
            "labels": labels
        })
    return samples

train_data = preprocess_train(train_raw)





In [ ]:
test_data = [
    {
        "id": x["ID"],
        "text": clean_text(x["Text"]),
        "tokens": zh_tokens(clean_text(x["Text"]))
    }
    for x in test_raw
]


In [ ]:
MODEL_NAME = "hfl/chinese-roberta-wwm-ext"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

label2id = {"O":0, "B-ASP":1, "I-ASP":2, "B-OPN":3, "I-OPN":4}
id2label = {v:k for k,v in label2id.items()}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
class DimASTEDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        enc = tokenizer(
            item["tokens"],
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )

        labels = [label2id[l] for l in item["labels"]]
        labels += [0] * (128 - len(labels))

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(labels[:128])
        }


In [ ]:
class DimASTEModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(MODEL_NAME)
        self.classifier = torch.nn.Linear(768, len(label2id))

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids, attention_mask)
        return self.classifier(out.last_hidden_state)


In [ ]:
model = DimASTEModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

class_weights = torch.tensor([0.02, 1.5, 1.5, 1.5, 1.5], device=device)

loader = DataLoader(DimASTEDataset(train_data), batch_size=8, shuffle=True)

for epoch in range(8):
    model.train()
    total_loss = 0

    for batch in tqdm(loader):
        optimizer.zero_grad()

        logits = model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device)
        )

        loss = torch.nn.functional.cross_entropy(
            logits.view(-1, len(label2id)),
            batch["labels"].to(device).view(-1),
            weight=class_weights
        )

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | BIO Loss: {total_loss/len(loader):.4f}")


pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

100%|██████████| 437/437 [01:27<00:00,  5.00it/s]


Epoch 1 | BIO Loss: 0.8090


100%|██████████| 437/437 [01:26<00:00,  5.05it/s]


Epoch 2 | BIO Loss: 0.6162


...

Epoch 6 | BIO Loss: 0.1470


100%|██████████| 437/437 [01:24<00:00,  5.18it/s]


Epoch 7 | BIO Loss: 0.1077


100%|██████████| 437/437 [01:24<00:00,  5.17it/s]

Epoch 8 | BIO Loss: 0.0838


In [ ]:
def aspect_pos_ok(span, doc):
    for s in doc.sentences:
        for w in s.words:
            if w.text in span and w.upos in {"NOUN","PROPN","VERB"}:
                return True
    return False


def opinion_pos_ok(span, doc):
    for s in doc.sentences:
        for w in s.words:
            if w.text in span:
                if w.upos in {"ADJ","ADV"}:
                    return True
                if len(span) == 1:   # IMPROVEMENT (2)
                    return True
                if w.upos == "VERB" and w.deprel not in {"aux"}:
                    return True
    return False


In [ ]:
def extract_triplets(text, tokens, labels):
    aspects, opinions = [], []
    cur, typ = [], None

    for t,l in zip(tokens, labels):
        if l.startswith("B-"):
            if cur:
                (aspects if typ=="ASP" else opinions).append("".join(cur))
            cur, typ = [t], l[2:]
        elif l.startswith("I-") and typ==l[2:]:
            cur.append(t)
        else:
            if cur:
                (aspects if typ=="ASP" else opinions).append("".join(cur))
            cur, typ = [], None
    if cur:
        (aspects if typ=="ASP" else opinions).append("".join(cur))

    doc = nlp_zh(text)

    aspects = [expand_aspect_np(a,doc) for a in aspects if aspect_pos_ok(a,doc)]
    opinions = [o for o in opinions if opinion_pos_ok(o,doc)]

    if aspects and not opinions:  # fallback
        for a in aspects:
            o = fallback_opinion(a,tokens,doc)
            if o:
                opinions.append(o)

    return [{"Aspect":a,"Opinion":o} for a in aspects for o in opinions]


In [ ]:
def expand_aspect_np(aspect, doc):
    for chunk in doc.noun_chunks:
        if aspect in chunk.text:
            return chunk.text
    return aspect


In [ ]:
tokenizer.add_tokens(["<ASP>", "</ASP>"])


0

In [ ]:
def fallback_opinion(aspect, tokens, doc, window=3):
    idxs = [i for i,t in enumerate(tokens) if t in aspect]
    for i in idxs:
        for j in range(max(0,i-window), min(len(tokens),i+window+1)):
            w = doc.sentences[0].words[j]
            if w.upos in {"ADJ","ADV","VERB"}:
                return tokens[j]
    return None


In [ ]:
def preprocess_zh_va(train_raw):
    rows = []
    for item in train_raw:
        text = clean_text(item["Text"])
        for q in item["Quadruplet"]:
            if q["Aspect"] == "NULL":
                continue
            v, a = map(float, q["VA"].split("#"))
            rows.append({
                "text": f"<ASP>{q['Aspect']}</ASP> {text}",
                "va": [v, a]
            })
    return rows

va_data = preprocess_zh_va(train_raw)


In [ ]:
class ChineseVADataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        enc = tokenizer(
            item["text"],
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["va"], dtype=torch.float)
        }


In [ ]:
class ChineseAspectVAModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(MODEL_NAME)
        self.encoder.resize_token_embeddings(len(tokenizer))
        self.regressor = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids, attention_mask)
        pooled = out.last_hidden_state[:, 0, :]
        return self.regressor(pooled)


In [ ]:
va_model = ChineseAspectVAModel().to(device)
optimizer = torch.optim.AdamW(va_model.parameters(), lr=2e-5)

loader = DataLoader(
    ChineseVADataset(va_data),
    batch_size=16,
    shuffle=True
)

for epoch in range(5):
    va_model.train()
    total_loss = 0
    for batch in tqdm(loader):
        optimizer.zero_grad()
        preds = va_model(
            batch["input_ids"].to(device),
            batch["attention_mask"].to(device)
        )
        loss = torch.nn.functional.smooth_l1_loss(
            preds, batch["labels"].to(device)
        )
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | VA Loss: {total_loss/len(loader):.4f}")


  7%|▋         | 27/407 [00:09<02:10,  2.91it/s]


KeyboardInterrupt: 

In [ ]:
def predict_va_zh(text, aspect):
    inp = f"<ASP>{aspect}</ASP> {text}"
    enc = tokenizer(inp, return_tensors="pt", truncation=True)

    with torch.no_grad():
        va = va_model(
            enc["input_ids"].to(device),
            enc["attention_mask"].to(device)
        )[0].cpu().numpy()

    va = np.clip(va, 1.0, 9.0)
    return f"{va[0]:.2f}#{va[1]:.2f}"


In [ ]:
model.eval()
va_model.eval()

with open("pred_zho_laptop.jsonl", "w", encoding="utf-8") as f:
    for item in test_data:
        enc = tokenizer(
            item["tokens"],
            is_split_into_words=True,
            return_tensors="pt",
            truncation=True
        )

        logits = model(
            enc["input_ids"].to(device),
            enc["attention_mask"].to(device)
        )

        probs = torch.softmax(logits, dim=-1)[0]
        labels = [id2label[p.argmax().item()] for p in probs]

        triplets = extract_triplets(
            item["text"],
            item["tokens"],
            labels
        )

        final_trips = []
        for t in triplets:
            va = predict_va_zh(item["text"], t["Aspect"])
            final_trips.append({
                "Aspect": t["Aspect"],
                "Opinion": t["Opinion"],
                "VA": va
            })

        f.write(json.dumps({
            "ID": item["id"],
            "Triplet": final_trips
        }, ensure_ascii=False) + "\n")

print("pred_zho_laptop.jsonl generated ✅")


AttributeError: 'Document' object has no attribute 'noun_chunks'

In [ ]:
# =========================================
# OFFICIAL cF1 VALIDATION (CHINESE – FIXED)
# =========================================

import re
import unicodedata
import numpy as np

D_MAX = 128.0  # (8^2 + 8^2)


# ---------- Normalization (CRITICAL FOR CHINESE) ----------
def normalize_zh(s):
    s = unicodedata.normalize("NFKC", s)
    s = s.replace(" ", "")
    s = re.sub(r"[，。！？；：、“”‘’（）《》【】]", "", s)
    return s


# ---------- VA distance ----------
def va_distance(pred_va, gold_va):
    return ((pred_va[0] - gold_va[0])**2 + (pred_va[1] - gold_va[1])**2) / D_MAX


def ctp(pred_va, gold_va):
    return max(0.0, 1.0 - va_distance(pred_va, gold_va))


# -------------------------------------------------
# Build GOLD triplets
# -------------------------------------------------
gold_triplets = {}

for item in train_raw:
    text = clean_text(item["Text"])
    trips = []

    for q in item["Quadruplet"]:
        if q["Aspect"] == "NULL":
            continue
        v, a = map(float, q["VA"].split("#"))
        trips.append({
            "Aspect": q["Aspect"],
            "Opinion": q["Opinion"],
            "VA": np.array([v, a])
        })

    gold_triplets[text] = trips


# -------------------------------------------------
# Predict triplets on DEV
# -------------------------------------------------
def predict_triplets_test(test_data):
    preds = {}

    model.eval()
    va_model.eval()

    with torch.no_grad():
        for item in test_data:
            text = item["text"]
            tokens = item["tokens"]

            enc = tokenizer(
                tokens,
                is_split_into_words=True,
                truncation=True,
                return_tensors="pt"
            )

            logits = model(
                enc["input_ids"].to(device),
                enc["attention_mask"].to(device)
            )

            probs = torch.softmax(logits, dim=-1)[0]
            labels = [id2label[p.argmax().item()] for p in probs]

            triplets = extract_triplets(text, tokens, labels)

            out = []
            for t in triplets:
                va_str = predict_va_zh(text, t["Aspect"])
                v, a = map(float, va_str.split("#"))
                out.append({
                    "Aspect": t["Aspect"],
                    "Opinion": t["Opinion"],
                    "VA": np.array([v, a])
                })

            preds[text] = out

    return preds


pred_triplets = predict_triplets_test(test_data)


# -------------------------------------------------
# Compute cF1 (NORMALIZED MATCHING)
# -------------------------------------------------
def compute_cF1(preds, golds):
    ctp_sum = 0.0
    pred_count = 0
    gold_count = 0

    for text in golds:
        gold_trips = golds[text]
        pred_trips = preds.get(text, [])

        gold_count += len(gold_trips)
        pred_count += len(pred_trips)

        used = set()

        for p in pred_trips:
            p_a = normalize_zh(p["Aspect"])
            p_o = normalize_zh(p["Opinion"])

            for i, g in enumerate(gold_trips):
                if i in used:
                    continue

                g_a = normalize_zh(g["Aspect"])
                g_o = normalize_zh(g["Opinion"])

                if p_a == g_a and p_o == g_o:
                    ctp_sum += ctp(p["VA"], g["VA"])
                    used.add(i)
                    break

    if pred_count == 0 or gold_count == 0:
        return 0.0, 0.0, 0.0

    cP = ctp_sum / pred_count
    cR = ctp_sum / gold_count
    cF1 = 2 * cP * cR / (cP + cR) if (cP + cR) > 0 else 0.0

    return cP, cR, cF1


# -------------------------------------------------
# Run evaluation
# -------------------------------------------------
cP, cR, cF1 = compute_cF1(pred_triplets, gold_triplets)

print("=== Subtask-2 Validation Performance (cF1) ===")
print(f"cPrecision : {cP:.4f}")
print(f"cRecall    : {cR:.4f}")
print(f"cF1        : {cF1:.4f}")

print("\nTriplet coverage:")
total_pred = sum(len(v) for v in pred_triplets.values())
total_gold = sum(len(v) for v in gold_triplets.values())
print("Predicted triplets:", total_pred)
print("Gold triplets     :", total_gold)
print("Recall ceiling    :", total_pred / total_gold if total_gold > 0 else 0)


=== Subtask-2 Validation Performance (cF1) ===
cPrecision : 0.0000
cRecall    : 0.0000
cF1        : 0.0000

Triplet coverage:
Predicted triplets: 283
Gold triplets     : 6502
Recall ceiling    : 0.04352506920947401


In [ ]:
from google.colab import files
files.download("pred_zho_laptop.jsonl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>